In [242]:
import torch
import torchvision

In [243]:
#batchsize
epochs=3
#parameter tuning:taring batchsize
train_batch = 64#default value
#train_batch = 128
#train_batch = 512
#train_batch = 1000
test_batch = 1000
#output a log for every 100 iterations
interval_print = 100

random_seed = 2022
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In this assignment, parameter tuning happens to these parameters: Batch size, hidden size, dropout,and optimizer<br>
batch size: 64 ,128, 512<br>
hidden size:5,10,20<br>
dropout : 0.3, 0.5,0.7<br>
Optimizer: SGD,Adam,Adagrad<br>


In [244]:
#dataset load
#since the global mean is 0.1307 and std deviation is 0.3081, we normalize the MNIST dataset using these two values with the builtin Normalize function
#load train data
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=train_batch, shuffle=True)
#load test data
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=test_batch, shuffle=True)

In [245]:
examples = enumerate(test_loader)
#get the batch index for the training
batch_idx, (example_data, example_targets) = next(examples)

In [246]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [247]:
class Pytorch_Net(nn.Module):
    def __init__(self):
        super(Pytorch_Net, self).__init__()
        #parameter tuning , hidden size 5,10,20
        self.input_hidden = nn.Conv2d(1, 10, kernel_size=5)
        self.hidden_output = nn.Conv2d(10, 20, kernel_size=5)#default value
        # self.input_hidden = nn.Conv2d(1, 5, kernel_size=5)
        # self.hidden_output = nn.Conv2d(5, 20, kernel_size=5)
        # self.input_hidden  = nn.Conv2d(1, 20, kernel_size=5)
        # self.hidden_output = nn.Conv2d(20, 20, kernel_size=5)

        #dropout layer
        self.dropout_layer = nn.Dropout2d()#p=0.5 default vakue
        #self.conv2_drop = nn.Dropout2d(p=0.3)
        #self.conv2_drop = nn.Dropout2d(p=0.7)
        #fully connected layers
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
      #use 2*2 pool kernel
        x = F.relu(F.max_pool2d(self.input_hidden(x), 2))
        x = F.tanh(F.max_pool2d(self.dropout_layer(self.hidden_output(x)), 2))
        #modify the shape
        x = x.view(-1, 320)
        #calculate through the fully connected layer
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [248]:
neural_net = Pytorch_Net()
optimizer=optim.Adam(neural_net.parameters(),lr=0.01,betas=(0.9,0.999))#default value
#optimizer = optim.SGD(neural_net.parameters(), lr=0.01,momentum=0.5)
#optimizer=optim.Adagrad(neural_net.parameters(),lr=0.01,lr_decay=0)

In [249]:
train_loss_total = []
train_iteration = []
test_loss_total = []
test_iteration = [i*len(train_loader.dataset) for i in range(epochs + 1)]

In [250]:
def train(epoch):
  #let the network start training
  neural_net.train()
  correct = 0
  #training for each batch
  for batch_idx, (data, target) in enumerate(train_loader):
    #initialize all gradient to zero
    optimizer.zero_grad()
    #get output
    output = neural_net(data)
    #calculate the loss function
    loss = F.nll_loss(output, target)
    loss.backward()
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).sum()
    optimizer.step()
    #output an result for certain interval
    if batch_idx % interval_print == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.0f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item(),100. * correct / (batch_idx * len(data))))
      train_loss_total.append(loss.item())
      train_iteration.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
  #print out overall training accuracy
  print('\nTrain set: Accuracy: {}/{} ({:.0f}%)\n'.format(
  correct, len(train_loader.dataset),
  100. * correct / len(train_loader.dataset)))

In [251]:
def test():
  #set module with evaluation mode
  neural_net.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():#disable the gradient calculation since it is the test phase, no further change should be made
    for data, target in test_loader:
      output = neural_net(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      #accumulate the predictions
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_loss_total.append(test_loss)
  #print out test accuracy
  print('\nTest set: Avg. loss: {:.2f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [252]:
def evaluation(n_epochs=epochs):#define how many runs to training
    #test the initial accuracy
    for epoch in range(1, n_epochs+1):
        train(epoch)
        test()

In [253]:
evaluation()

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1949: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.357809	Accuracy: inf
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.722148	Accuracy: 61
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.372094	Accuracy: 71
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.552096	Accuracy: 74
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.389269	Accuracy: 76
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.308666	Accuracy: 78
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.375377	Accuracy: 79
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.706982	Accuracy: 80
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.190001	Accuracy: 80
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.506243	Accuracy: 81

Train set: Accuracy: 48681/60000 (81%)



/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.19, Accuracy: 9559/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.217066	Accuracy: inf
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.642016	Accuracy: 88
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.481078	Accuracy: 87
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.821658	Accuracy: 87
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.450156	Accuracy: 87
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.312484	Accuracy: 87
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.265387	Accuracy: 87
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.624976	Accuracy: 87
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.181982	Accuracy: 87
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.333581	Accuracy: 87

Train set: Accuracy: 52120/60000 (87%)


Test set: Avg. loss: 0.20, Accuracy: 9566/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.324862	Accuracy: inf
Train Epoch: 3 [6400/60000 (11%)]	Loss: 0.424257	Accuracy: 88
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.284344	Accuracy: 87
Train Epoch: 3 [19200/60000 (32%)]	Los